In [5]:
import os

In [6]:
from narrative_llm_agent.agents.kbase_agent import KBaseAgent
from narrative_llm_agent.agents.analyst import AnalystAgent
from narrative_llm_agent.agents.KnowledgeGraph import KGAgent
from langchain_openai import ChatOpenAI
from crewai import Task, Crew

In [7]:
api_key = os.environ["OPENAI_API_KEY"]
neo4j_uri = os.environ["NEO4J_URI"]
neo4j_username = os.environ["NEO4J_USERNAME"]
neo4j_password = os.environ["NEO4J_PASSWORD"]

In [8]:
token = "dumy_token"

In [9]:
def test_kbase_agent():
    agent = KBaseAgent("foo", None)
    assert agent._token == "foo"
    print("like totally passed.")

In [10]:
llm = ChatOpenAI(temperature=0, model="gpt-4", streaming=True)

In [11]:
researcher = AnalystAgent(token=token, llm=llm)
KGer = KGAgent(token=token, llm=llm)

In [8]:
# # Create tasks for your agents
# task1 = Task(
#   description="""I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing.
#   Using KBase apps, I would like to understand if my newly sequenced microbe is a cysteine auxotroph. Which KBase apps should I use and how do I use them?
#   Use the Kbase App catalog and documentation to come up with a list of apps to use for each step.
#   Use the knowledge graph to make sure that the apps that you suggest exist
#   Get the approval of a human.
# """,
#   agent=researcher.agent
# )
# task2 = Task(
#   description="""I am trying to use the trimmomatic and gtdb apps, are there any common data objects between the apps?
# """,
#   agent=researcher.agent
# )

In [14]:
# task1 = Task(
#   description="""I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing.
#   Using KBase apps, I would like to understand if my newly sequenced microbe is a cysteine auxotroph. Which KBase apps should I use and how do I use them?
#   """,
#   agent=researcher.agent
# )
task1 = Task(
    description="""I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing tand would like to check its quality.
    What app should I use. Please suggest only one app at a time for a particular step. If you must suggest alternative apps please limit to only one.""",
    agent=researcher.agent,
)
task2 = Task(
    description="""
  Use the knowledge graph to make sure that the apps that you suggest exist
  After you check its availability in the Knowledge Graph, ask a human if they approve of the suggested app and then only return the app ID. If not check for the additional recommended app in the Knowledge Graph.""",
    agent=KGer.agent,
)

In [15]:
# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher.agent, KGer.agent],
    tasks=[task1, task2],
    verbose=2,  # You can set it to 1 or 2 to different logging levels
)

Overriding of current TracerProvider is not allowed


In [18]:
# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

[DEBUG]: Working Agent: Computational Biologist and Geneticist
[INFO]: Starting Task: I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing tand would like to check its quality.
    What app should I use. Please suggest only one app at a time for a particular step. If you must suggest alternative apps please limit to only one.


> Entering new CrewAgentExecutor chain...
Use Tool: KBase app catalog retrieval tool
Input: "What is the best app in KBase for checking the quality of a fastq file from paired end illumina sequencing?" 

I already used the kbase app catalog retrieval tool tool with input What is the best app in KBase for checking the quality of a fastq file from paired end illumina sequencing?. So I already know that and must stop using it with same input. 
I could give my best complete final answer if I'm ready, using exaclty the expected format bellow:

```
Final Answer: [my expected final answer, entire content of my most complete f

 Assess Read Quality with FastQC - v0.12.1


 

Assess Read Quality with FastQC - v0.12.1

Use Tool: KG retrieval tool
Input: "Assess Read Quality with FastQC - v0.12.1"

> Entering new AgentExecutor chain...

Invoking: `Information` with `{'entity': 'Assess Read Quality with FastQC - v0.12.1', 'entity_type': 'AppCatalog'}`


candidate name provided= [{'candidate': 'Assess Read Quality with FastQC - v0.12.1'}]
type:App
Input Data Object Type: KBaseAssembly.PairedEndLibrary, KBaseAssembly.SingleEndLibrary, KBaseFile.PairedEndLibrary, KBaseFile.SingleEndLibrary, KBaseRNASeq.RNASeqSampleSet

App name: Assess Read Quality with FastQC - v0.12.1
Tooltip: A quality control application for high throughput sequence data.
AppID: kb_fastqc/runFastQCThe "Assess Read Quality with FastQC - v0.12.1" is a quality control application for high throughput sequence data. It accepts the following input data object types: 

- KBaseAssembly.PairedEndLibrary
- KBaseAssembly.SingleEndLibrary
- KBaseFile.PairedEndLibrary
- KBaseFile.SingleEndLibrary
- KBa